In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from scipy.stats import kurtosis
import pandas as pd
import numpy as np
import cv2

In [ ]:
BASE = "/content/drive/MyDrive/CAR DENT PROJECT/Dent 0_1 dataset/"

CSV_PATH = BASE + "dataset 0_1.csv"
IMG_DIR = BASE + "images"

df = pd.read_csv(CSV_PATH)
print("Total samples:", len(df))

Total samples: 91


In [ ]:
df.head()

,image,label
0,01.jpg,0
1,02.jpg,0
2,03.jpg,0
3,04.jpg,0
4,05.jpg,0


In [ ]:
def compute_aux_labels(img_bgr):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    v = hsv[:, :, 2]

    gx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
    grad_mag = np.sqrt(gx**2 + gy**2)

    # Bright region
    bright_ratio = np.mean(v > 230)
    high_brightness_region = int(bright_ratio > 0.02)

    # Specular highlight
    specular_mask = (v > 240) & (grad_mag < np.mean(grad_mag))
    has_specular_highlight = int(np.mean(specular_mask) > 0.005)

    # Sharp intensity peak
    hist = cv2.calcHist([gray], [0], None, [256], [0,256]).flatten()
    hist = hist / (hist.sum() + 1e-6)
    sharp_intensity_peak = int(kurtosis(hist) > 8)

    # Depth gradient (dent cue)
    grad_var = np.var(grad_mag)
    has_depth_gradient = int(grad_var > 50)

    # Radial shading
    h, w = gray.shape
    yy, xx = np.indices((h, w))
    r = np.sqrt((xx - w/2)**2 + (yy - h/2)**2)
    r /= (r.max() + 1e-6)
    corr = np.corrcoef(r.flatten(), gray.flatten())[0, 1]
    has_radial_shading = int(abs(corr) > 0.15)

    return (
        high_brightness_region,
        has_specular_highlight,
        sharp_intensity_peak,
        has_depth_gradient,
        has_radial_shading
    )

In [ ]:
df[
    [
        "high_brightness_region",
        "has_specular_highlight",
        "sharp_intensity_peak",
        "has_depth_gradient",
        "has_radial_shading"
    ]
] = 0  # initialize columns

In [ ]:
for idx, row in df[:].iterrows():
    img_path = f"{IMG_DIR}/{row['image']}"
    img = cv2.imread(img_path)

    if img is None:
        print(f"Warning: could not load {img_path}")
        continue

    aux = compute_aux_labels(img)
    df.loc[idx, [
        "high_brightness_region",
        "has_specular_highlight",
        "sharp_intensity_peak",
        "has_depth_gradient",
        "has_radial_shading"
    ]] = aux

In [ ]:
df

,image,label,high_brightness_region,has_specular_highlight,sharp_intensity_peak,has_depth_gradient,has_radial_shading
0,01.jpg,0,0,0,1,1,1
1,02.jpg,0,0,0,1,1,1
2,03.jpg,0,0,0,1,1,1
3,04.jpg,0,1,0,1,1,0
4,05.jpg,0,0,0,1,1,0
...,...,...,...,...,...,...,...
86,87.jpg,1,0,0,1,1,0
87,88.jpg,1,0,0,1,1,0
88,89.jpg,1,0,0,1,1,0
89,90.jpg,1,0,0,1,1,1


# **MODEL WITH AUX LABELS**

In [ ]:
import torch
from torch.utils.data import Dataset

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(
        brightness=0.1,
        contrast=0.1
    ),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
import torch
from torch.utils.data import Dataset
import cv2
import numpy as np

class DentDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform

        self.aux_cols = [
            "high_brightness_region",
            "has_specular_highlight",
            "sharp_intensity_peak",
            "has_depth_gradient",
            "has_radial_shading"
        ]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]

        # ---- Image ----
        img = cv2.imread(f"{self.image_dir}/{row['image']}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transform:
            img = self.transform(img)
        else:
            img = transforms.ToTensor()(img)

        # ---- Aux labels as INPUT ----
        aux = torch.tensor(
            row[self.aux_cols].values.astype(float), # Ensure numeric type
            dtype=torch.float32
        )

        # ---- Target ----
        label = torch.tensor(row["label"], dtype=torch.float32)

        return img, aux, label


In [ ]:
train_dataset = DentDataset(
    train_df,
    image_dir=IMG_DIR,
    transform=train_transform
)

val_dataset = DentDataset(
    val_df,
    image_dir=IMG_DIR,
    transform=val_transform
)

In [ ]:
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader # Added DataLoader import


In [ ]:
class DentResNetWithAux(nn.Module):
    def __init__(self, num_aux, pretrained=True):
        super().__init__()

        # ---- Load pretrained ResNet ----
        self.backbone = models.resnet18(pretrained=pretrained)

        # Remove final FC
        self.backbone.fc = nn.Identity()   # output: 512

        # ---- Image feature projection ----
        self.fc_img = nn.Linear(512, 128)

        # ---- Aux feature branch ----
        self.fc_aux = nn.Sequential(
            nn.Linear(num_aux, 32),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # ---- Final classifier ----
        self.fc_out = nn.Linear(128 + 32, 1)

    def forward(self, img, aux):
        img_feat = self.backbone(img)      # (B, 512)
        img_feat = self.fc_img(img_feat)   # (B, 128)

        aux_feat = self.fc_aux(aux)        # (B, 32)

        fused = torch.cat([img_feat, aux_feat], dim=1)
        out = self.fc_out(fused)

        return out

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = DentResNetWithAux(num_aux=5).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam([
    {"params": model.backbone.parameters(), "lr": 1e-4},
    {"params": model.fc_img.parameters(), "lr": 1e-3},
    {"params": model.fc_aux.parameters(), "lr": 1e-3},
    {"params": model.fc_out.parameters(), "lr": 1e-3}
])


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 230MB/s]


In [ ]:
for p in model.backbone.parameters():
    p.requires_grad = False

In [ ]:
batch_size = 32 # Define a batch size

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

epochs = 40
best_loss = float("inf")
for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for img, aux, label in train_loader: # Iterate over DataLoader
        img, aux, label = img.to(device), aux.to(device), label.to(device)

        optimizer.zero_grad()
        output = model(img, aux) # Remove .unsqueeze(0) as DataLoader handles batch dimension
        loss = criterion(output, label.unsqueeze(1)) # Adjust label shape for BCEWithLogitsLoss
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss /= len(train_loader) # Divide by number of batches

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for img, aux, label in val_loader: # Iterate over DataLoader
            img, aux, label = img.to(device), aux.to(device), label.to(device)
            output = model(img, aux) # Remove .unsqueeze(0) as DataLoader handles batch dimension
            loss = criterion(output, label.unsqueeze(1)) # Adjust label shape for BCEWithLogitsLoss
            val_loss += loss.item()
        val_loss /= len(val_loader) # Divide by number of batches
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")


Epoch 1/40 | Train Loss: 0.8391 | Val Loss: 0.5711
Epoch 2/40 | Train Loss: 0.7692 | Val Loss: 0.5156
Epoch 3/40 | Train Loss: 0.5125 | Val Loss: 0.4298
Epoch 4/40 | Train Loss: 0.3833 | Val Loss: 0.3400
Epoch 5/40 | Train Loss: 0.4155 | Val Loss: 0.2758
Epoch 6/40 | Train Loss: 0.2533 | Val Loss: 0.2473
Epoch 7/40 | Train Loss: 0.2410 | Val Loss: 0.2241
Epoch 8/40 | Train Loss: 0.1960 | Val Loss: 0.1972
Epoch 9/40 | Train Loss: 0.1885 | Val Loss: 0.1891
Epoch 10/40 | Train Loss: 0.1585 | Val Loss: 0.1713
Epoch 11/40 | Train Loss: 0.1164 | Val Loss: 0.1792
Epoch 12/40 | Train Loss: 0.2372 | Val Loss: 0.1660
Epoch 13/40 | Train Loss: 0.1326 | Val Loss: 0.1404
Epoch 14/40 | Train Loss: 0.0896 | Val Loss: 0.1280
Epoch 15/40 | Train Loss: 0.1077 | Val Loss: 0.1296
Epoch 16/40 | Train Loss: 0.0905 | Val Loss: 0.1621
Epoch 17/40 | Train Loss: 0.0717 | Val Loss: 0.2359
Epoch 18/40 | Train Loss: 0.2645 | Val Loss: 0.1641
Epoch 19/40 | Train Loss: 0.1720 | Val Loss: 0.0982
Epoch 20/40 | Train L